In [1]:
import os
os.environ["PATH"] = '/home/mattjones/bin:/home/mattjones/.local/bin:/home/mattjones/myapps/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/mattjones/opt/gurobi801/linux64/bin:/home/mattjones/software/bowtie2-2.3.4.2:/home/mattjones/emboss/EMBOSS-6.6.0/emboss'
os.environ["LD_LIBRARY_PATH"] = '/home/mattjones/lib/::/home/mattjones/opt/gurobi801/linux64/lib'

from Cassiopeia.TreeSolver import data_pipeline
from Cassiopeia.TreeSolver.simulation_tools import dataset_generation
from Cassiopeia.TreeSolver import score_triplets

import pandas as pd 
import numpy as np
import networkx as nx
import pickle as pic

import sys

### Simulate Tree

In [2]:
# set parameters
D = 12
C = 40
S = 20
num_cells = 400
sub_samp = num_cells / 2**D
M = 0.0275

dropout = dict(zip(range(C), np.random.randint(7, high=40, size=C) / 100))

# create mutation map
mutation_map = {}
for i in range(C):
    sampled_probabilities = sorted([np.random.negative_binomial(5, .5) for _ in range(1, S)])
    mutation_map[i] = {'0': (1 - M)}
    total = np.sum(sampled_probabilities)
    for j in range(1, S):
        mutation_map[i][str(j)] = (M)*sampled_probabilities[j-1]/(1.0 * total)

In [3]:
true_network = dataset_generation.generate_simulated_full_tree(mutation_map, dropout, characters=C, subsample_percentage = sub_samp, depth=D)

pic.dump(true_network, open("true_network_test.pkl", "wb"))

### Reconstruct Trees with the Stress-Test Module

You can use the `stress-test` command line tool to reconstruct the trees from the simulation. This module will take in the newtorkx file produced from the `generate_simulated_full_tree` function and reconstruct a tree using any of five algorithms. Use the `-h` flag to see usage and all options.

Below, we have used already run the stress test command on the true network and have saved the result in `greedy_network_test.pkl`.

### Validation

In [4]:
reconstructed_tree = nx.read_gpickle("greedy_network_test.pkl")
true_network = nx.read_gpickle("true_network_test.pkl")

triplet_score = score_triplets.score_triplets(true_network, reconstructed_tree, modified=True, number_of_trials=10000)

print("Triplet score: " + str(triplet_score))

Triplet score: 0.8243561501183451
